## <div style="text-align: right">   第6章 </div>

##  <div style="text-align: right"> 进程皆有资源限制 </div>

___

在上一章我们了解到了这样一个事实：文件描述符代表已打开的资源。你可能注意到当资源没有被关闭时，文件描述符一直处于递增的状态。这就产生一个问题：一个进程可以拥有多少个文件描述符？

答案取决于你的系统配置，不过重要的一点是：**内核为进程施加了某些资源限制**。

### 6.1 找出限制

让我们继续文件描述符的话题。我们可以使用python来查询所允许的最大的文件描述符编号：

In [1]:
import psutil
p = psutil.Process()
p.rlimit(psutil.RLIMIT_NOFILE)

(1024, 4096)

我们传入一个常量PLIMIT_NOFILE到进程的rlimit方法中查询进程
所能打开的最大文件数。它返回了一个包含两个元素的元组。

元组的第一个元素是文件描述符的软限制（soft limit），第二个元素是文件描述的硬限制（hard limit）。

### 6.2 软限制与硬限制

两者有什么不同吗？问的好，软限制其实算不上一种限制。也就是说如果超出了软限制（在这里指一次性打开超过1024个资源），将会产生异常。不过只要你愿意，就可以修改这个限制。

所以说任何进程都可以修改自身的软限制，但是对于硬限制呢？通称情况下，只有超级用户能够修改硬限制。不过如果进程有足够的权限，
那么也可以修改硬限制。如果你对更改系统级别的限制感兴趣，可以查看 _sysctl(8)_ 。

### 6.3  提高软限制

让我们试着提高当前的软限制：

In [2]:
p.rlimit(psutil.RLIMIT_NOFILE, (4096, 4096))
p.rlimit(psutil.RLIMIT_NOFILE)

(4096, 4096)

可以看到，我们为可打开的文件数量设置了一个新限制，
再次查询发现软限制和硬限制都设成了新值4096。

如果我们有足够的权限，我们便可以提高进程的硬限制。但反过来，
降低进程的硬限制并不需要什么权限，但要注意对硬限制的
降低是不可逆转的——一旦调低，就没法再调高。

In [4]:
p.rlimit(psutil.RLIMIT_NOFILE, (128, 128))
p.rlimit(psutil.RLIMIT_NOFILE)

(128, 128)

In [7]:
p.rlimit(psutil.RLIMIT_NOFILE, (1024, 1024))

AccessDenied: psutil.AccessDenied (pid=28619)

可以看到，当我们将进程硬限制降低到128后，如果想再次调高到1024,便会产生异常，只能不断的降低。

In [8]:
p.rlimit(psutil.RLIMIT_NOFILE, (100, 100))
p.rlimit(psutil.RLIMIT_NOFILE)

(100, 100)

接下来的例子是一个将系统资源的软限制提高至硬限制水平，即所允许的最大值的常用方法。

In [9]:
max_nofile = p.rlimit(psutil.RLIMIT_NOFILE)[1]
p.rlimit(psutil.RLIMIT_NOFILE, (max_nofile, max_nofile))

### 6.4 超出限制

要注意的是，如果超出了限制，则会抛出异常。

In [2]:
# 将可以打开的最大文件数设置为3, 我们知道这会超出最大限制，
# 因为标准流已经占用了前三个文件描述符。
p.rlimit(psutil.RLIMIT_NOFILE, (4,4))
new_file = open('new.txt', 'w')

Exception in thread IPythonHistorySavingThread:
Traceback (most recent call last):
  File "/home/xiaojia/anaconda3/lib/python3.7/site-packages/IPython/core/history.py", line 834, in run
  File "<decorator-gen-23>", line 2, in writeout_cache
  File "/home/xiaojia/anaconda3/lib/python3.7/site-packages/IPython/core/history.py", line 58, in needs_sqlite
  File "/home/xiaojia/anaconda3/lib/python3.7/site-packages/IPython/core/history.py", line 780, in writeout_cache
  File "/home/xiaojia/anaconda3/lib/python3.7/site-packages/IPython/core/history.py", line 764, in _writeout_input_cache
sqlite3.OperationalError: unable to open database file

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/xiaojia/anaconda3/lib/python3.7/site-packages/ipykernel/iostream.py", line 97, in _event_pipe
AttributeError: '_thread._local' object has no attribute 'event_pipe'

During handling of the above exception, another exception occurred:

Trace

OSError: [Errno 24] Too many open files: 'new.txt'

> 注意到，在上述命令中，即便我们将软限制设置为4,新打开文件时仍然会产生异常，这是因为jupyter-notebook
 进程自己启动时会打开多个文件用于提供相关服务，因此此时已经打开的文件数量不仅仅包括3个标准流文件

### 6.5 其他资源

你可以用了类似的方法来检查及修改其他的资源限制，通过向rlimit方法中传入其他类型的常量，一些常见的例子如下：

In [3]:
# 当前用户所允许的最大并发进程数
print(p.rlimit(psutil.RLIMIT_NPROC))

# 可以创建的最大的文件
print(p.rlimit(psutil.RLIMIT_FSIZE))

# 用于进程栈的最大段的大小
print(p.rlimit(psutil.RLIMIT_STACK))

(47647, 47647)
(-1, -1)
(8388608, -1)


上述命令中，文件大小查询返回（-1, -1），表示文件可以无限大
（在硬件资源允许的情况下），关于更多的资源限制，可以查看
[prlimit(2)](https://linux.die.net/man/2/prlimit)。

### 6.6 实践领域

多数程序通常并不需要修改系统资源限制。但对于一些特殊的工具，这可是至关重要的。

其中一种情况是某个进程需要处理成千上万的并发网络连接。例如http
性能测试工具 _httperf(1)_ 。
像`httperf --hog --server www --num-conn 5000`这样的命令
会使得httperf() 创建5000个并发连接。由于默认的软限制，我的系统显然会出现问题。因此在进行正确的测试之前，httperf(1) 需要调高对应的软限制。

另一个需要进行系统资源限制的实际用例是，在执行第三法代码并需要对其施加一些运行限制时。你可以对代码所属的进程设置限制，并取消修改这些限制的权限，这样就能确保其无法使用超出许可范围的资源数量。

### 6.7 系统调用

python中进程对象系统资源限制的访问和调整分别对应于
_getrlimit(2)_ 及 _setrlimit(2)_ 。